In [1]:
%%bash

pip install scrapy --q

In [3]:
%%bash
cat > myspider.py <<EOF
import scrapy

class MirrorSpider(scrapy.Spider):
    name = 'spyder'
    start_urls = ['PAST_LINK_HERE'] #https://www.dailymirror.lk/latest-news/108

    def parse(self, response):
        # Extract titles, content, and datetime
        for news_item in response.xpath('//*[@id="breakingnewsads"]/div[1]/div/div[1]/a'):
            title = news_item.css('.cat-hd-tx::text').get()
            content = news_item.xpath('p[2]/text()').get()
            datetime = news_item.css('.gtime::text').get()
            if title and content and datetime:
                yield {
                    'title': title.strip(),
                    'content': content.strip(),
                    'datetime': datetime.strip()
                }

        # Follow pagination links if available
        next_page = response.css('.pagination li:last-child a::attr(href)').get()
        if next_page:
            yield response.follow(next_page, self.parse)
EOF

In [4]:
!scrapy runspider myspider.py -o daily-mirror.json --nolog